# KNN

## KNN介绍

1．$k$近邻法是基本且简单的分类与回归方法。$k$近邻法的基本做法是：对给定的训练实例点和输入实例点，首先确定输入实例点的$k$个最近邻训练实例点，然后利用这$k$个训练实例点的类的多数来预测输入实例点的类。

2．$k$近邻模型对应于基于训练数据集对特征空间的一个划分。$k$近邻法中，当训练集、距离度量、$k$值及分类决策规则确定后，其结果唯一确定。

3．$k$近邻法三要素：距离度量、$k$值的选择和分类决策规则。常用的距离度量是欧氏距离及更一般的**pL**距离。$k$值小时，$k$近邻模型更复杂；$k$值大时，$k$近邻模型更简单。$k$值的选择反映了对近似误差与估计误差之间的权衡，通常由交叉验证选择最优的$k$。

常用的分类决策规则是多数表决，对应于经验风险最小化。

4．$k$近邻法的实现需要考虑如何快速搜索k个最近邻点。**kd**树是一种便于对k维空间中的数据进行快速检索的数据结构。kd树是二叉树，表示对$k$维空间的一个划分，其每个结点对应于$k$维空间划分中的一个超矩形区域。利用**kd**树可以省去对大部分数据点的搜索， 从而减少搜索的计算量。

### 距离度量

设特征空间$x$是$n$维实数向量空间 ，$x_{i}, x_{j} \in \mathcal{X}$,$x_{i}=\left(x_{i}^{(1)}, x_{i}^{(2)}, \cdots, x_{i}^{(n)}\right)^{\mathrm{T}}$,$x_{j}=\left(x_{j}^{(1)}, x_{j}^{(2)}, \cdots, x_{j}^{(n)}\right)^{\mathrm{T}}$
，则：$x_i,x_j$的$L_p$距离定义为:


$L_{p}\left(x_{i}, x_{j}\right)=\left(\sum_{i=1}^{n}\left|x_{i}^{(i)}-x_{j}^{(l)}\right|^{p}\right)^{\frac{1}{p}}$

- $p= 1$  曼哈顿距离
- $p= 2$  欧氏距离
- $p= \infty$   切比雪夫距离

## 1、数据预处理
* 导入需要的库
* 导入数据
* 检查数据缺失值，Onehotencoder编码是对离散特征值的编码方式，用于给线性模型增加非线性能力。
* 划分数据集
* 特征工程

In [9]:
import numpy as np 
import pandas as pd
from sklearn.datasets import load_iris  #数据集
from sklearn.model_selection import train_test_split #数据划分
from sklearn.preprocessing import StandardScaler
#导入数据集
iris=load_iris()
df=pd.DataFrame(iris.data,columns=iris.feature_names)
df["label"]=iris.target
x=df.iloc[:,:2].values
y=df.iloc[:,-1].values
#划分数据集
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=0)
#特征工程:由于数据并没有过大的差值，本实例不进行特征工程
sd=StandardScaler()
x_train=sd.fit_transform(x_train)
x_test=sd.transform(x_test)

## 模型训练
* sklearn.neighbors库的KNeighborsClassifier类。
* 创建一个KNeighborsClassifier类的**classifier**对象。
* 使用KNeighborsClassifier类的fit()方法将**classifier**对象对数据集进行训练。

In [10]:
from sklearn.neighbors import KNeighborsClassifier
classifier=KNeighborsClassifier()
classifier.fit(x_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

## 模型预测



In [11]:
#测试集
y_pred=classifier.predict(x_test)    #预测结果
print(y_pred)
print("Compare actual results with predicted results:\n",y_pred==y_test)  

[1 1 0 2 0 2 0 1 2 2 1 2 2 2 2 0 2 1 0 0 1 1 0 0 2 0 0 2 1 0]
Compare actual results with predicted results:
 [False  True  True  True  True  True  True  True False False False False
 False False False  True False  True  True  True False  True  True  True
  True  True  True False  True  True]


## 模型评估

In [12]:
#测试集
score=classifier.score(x_test,y_test)  #计算准确率
print(f"Accuracy={score*100}%")  #Calculate accuracy

Accuracy=63.33333333333333%


## 优化

如何确定最佳k值? 因此我们运用到交叉验证。
* sklearn.model_selection.GridSearchCV(estimator,param_grid=None,cv=None)
* param_grid: Estimator parameters(dict){"n_neighbors:[1,4,6]"}
* cv=4: 4 fold cross-validation(4折交叉验证) 

In [18]:
from sklearn.model_selection import GridSearchCV
#模型训练
estimator=KNeighborsClassifier() #可以输入k值
'''several fold cross-validation'''
param_dict={"n_neighbors":[4,3,6,7,8]} #K-value gird search（k值网格搜索）
estimator=GridSearchCV(estimator=estimator, #优化器
                        param_grid=param_dict, #网格搜索k值
                        cv=5  #几折交叉验证
                        )
estimator.fit(x_train,y_train)  #模型训练
print(estimator)
#模型预测
test_pred=estimator.predict(x_test)    #预测结果
print(test_pred)
print("Compare actual results with predicted results:\n",test_pred==y_test)  
#模型评估
score=estimator.score(x_test,y_test)  #计算准确率
print(f"Accuracy={score*100}%")  #Calculate accuracy

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [4, 3, 6, 7, 8]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
[0 1 0 1 1 0 0 1 0 1 0 0 0 1 1 0 0 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 0 1 0 0 0
 1 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1 1 0 0 1 1 0 0 0
 1 0 0 1 1 0]
Compare actual results with predicted results:
 [ True  True  True  True  True  True  True  True  True  True  True False
  True  True  True  True  True  True  True  True  True  True  True  True
  True False  True  True  True  True  True  T

## 模型保存
* sklearn.externals import joblib【知道】
* 保存：joblib.dump(estimator, 'test.pkl')
* 加载：estimator = joblib.load('test.pkl')


In [20]:
from sklearn.externals import joblib  
from sklearn.preprocessing import  StandardScaler

#导入数据
data=pd.read_csv(r"D:\program Lab Report\machine learning\100day\datasets\Social_Network_Ads.csv")
x=data.iloc[:,[2,3]].values
y=data.iloc[:,-1].values
#数据划分
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)
print(len(x_train))
print(len(x_test))
#特征工程
sd=StandardScaler()
x_train=sd.fit_transform(x_train)
x_test=sd.transform(x_test)

#导入模型
best_estimator=joblib.dump(estimator,r"knn_test.pkl")  #模型保存
estimator=joblib.load(r"D:\program Lab Report\machine learning\100day\code\4_kNN\knn_test.pkl")
print(estimator)

#模型预测
test_pred=estimator.predict(x_test)    #预测结果
print(test_pred)

#模型评估
score=estimator.score(x_test,y_test)  #计算准确率
print(f"Accuracy={score*100}%")  #Calculate accuracy

320
80
GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [4, 3, 6, 7, 8]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)
[1 0 0 1 0 0 1 0 1 0 1 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 1 0 1 0 1 1 1
 1 1 1 0 1 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 1 1 0 0 0 0 1 0 1 0 1 0 0 0 0 0 1
 1 0 0 0 0 0]
Accuracy=93.75%
